<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/course2/week1/Regularization_v2a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regularization(정규화) #

금주의 두 번째 과제에 오신것을 환영합니다! 딥 러닝 모델은 유연성과 용량이 매우 커서, 훈련 데이터셋이 충분히 크지 않을 경우 **과적합이 심각한 문제가 될 수 있을 정도입니다.** 훈련된 인공 신경먕은 훈련 세트에서는 잘 작동하겠지만, 한 번도 본적 없는 새로운 형태의 예제가 올 경우에도 잘 동작할 수 있도록 **일반화되지 않았습니다.**

**이번 과제에서는,** 지금까지 개발한 인공신경망에 정규화를 적용해보겠습니다.

먼저 과제를 하는 동안 필요한 패키지를 다운로드받아봅시다.




In [ ]:
# import packages
import numpy as np
import matplotlib.pyplot as plt
from reg_utils import sigmoid, relu, plot_decision_boundary, initialize_parameters, load_2D_dataset, predict_dec
from reg_utils import compute_cost, predict, forward_propagation, backward_propagation, update_parameters
import sklearn
import sklearn.datasets
import scipy.io
from testCases import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (7.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

**문제 정의하기**: 지금부터 당신이 프랑스 축구협회의 AI 전문가로 고용되었다고 상상해봅시다. 그들은 당신이 프랑스 선수들이 헤딩을 할 수 있도록 프랑스 골키퍼가 골킥을 차야할 위치를 추천하길 원합니다.

<img src="arts/field_kiank.png" style="width:600px;height:350px;">
<caption><b>그림1</b> : <b>축구장</b></caption>

<br>

골키퍼가 공중으로 공을 차면, 각 팀의 선수들이 그 공을 헤딩하기 위해 싸우고 있습니다.

이 문제와 함께, 프랑스 축구협회는 프랑스 축구 대표팀의 지난 10개 경기를 담은 2D 데이터 셋을 제공해주었습니다.


In [ ]:
train_X, train_Y, test_X, test_Y = load_2D_dataset()

각각의 점은 프랑스 골키퍼가 축구장 왼쪽으로부터 찬 공에 축구 선수가 헤딩을 성공시킨 축구장 내 위치에 해당합니다.
- 파란 색 점은 프랑스 선수가 성공시킨 헤딩의의 위치입니다.
- 빨간 색 점은 상대방 선수가 성공시킨 헤딩의 위치입니다.

**목표**: 딥 러닝 모델을 이용하여, 축구장 내에서 골키퍼가 공을 찰 수 있는 가장 효율적인 위치를 찾아보세요


**데이터셋 분석**: 이 데이터셋은 약간 매끄럽지 않아보이지만, 전반적으로 그래프에서 좌상향하는 직선을 그리면 빨간 점과 파란 점을 구분할 수 있는 것 같아 보입니다.

먼저 정규화되지 않은 모델을 사용해서 예측해보겠습니다. 그리고 나서, 어떻게 정규화를 진행하는지, 그리고 이 프랑스 축구 협회 문제를 해결하기 위해서는 어떤 모델을 사용해야 하는지 차차 다루어보도록 하겠습니다.

## 1. Non-regularized model (비 정규화 모델) ##

아래 코드 블록에 이미 구현되어 있는 인공 신경망을 사용합니다. 이 모델은 아래와 같은 방법들로 사용될 수 있습니다.
- `regularization` 모드 : `lambd` 변수를 0이 아닌 값으로 설정합니다. `lambda`는 파이썬에서 예약어로 쓰이기 때문에 `lambd`라는 이름으로 변수를 만들었습니다.
- `dropout` 모드 : `keep_prob` 변수를 0보다 작은 값으로 설정합니다.

어떤 정규화도 하지 않고 모델을 학습해보겠습니다. 그리고 나서, 아래 두 방법을 시도할 것입니다.
- *L2 정규화*: `compute_cost_with_regularization()` 함수와 `backward_propagation_with_regularization()` 함수를 사용합니다.
- *Dropout* : `forward_propagation_with_dropout()` 함수와 `backward_propagation_with_dropout()` 함수를 사용합니다.

각 파트에서, 적절한 입력값으로 이 모델을 돌려 여러분이 구현한 함수를 호출하도록 할 것입니다. 아래 코드 블록을 살펴보고 모델 구조에 익숙해지도록 하세요.

In [ ]:
def model(X, Y, learning_rate = 0.3, num_iterations = 30000, print_cost = True, lambd = 0, keep_prob = 1):
    """
    Implements a three-layer neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SIGMOID.
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (1 for blue dot / 0 for red dot), of shape (output size, number of examples)
    learning_rate -- learning rate of the optimization
    num_iterations -- number of iterations of the optimization loop
    print_cost -- If True, print the cost every 10000 iterations
    lambd -- regularization hyperparameter, scalar
    keep_prob - probability of keeping a neuron active during drop-out, scalar.
    
    Returns:
    parameters -- parameters learned by the model. They can then be used to predict.
    """
        
    grads = {}
    costs = []                            # to keep track of the cost
    m = X.shape[1]                        # number of examples
    layers_dims = [X.shape[0], 20, 3, 1]
    
    # Initialize parameters dictionary.
    parameters = initialize_parameters(layers_dims)

    # Loop (gradient descent)

    for i in range(0, num_iterations):

        # Forward propagation: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID.
        if keep_prob == 1:
            a3, cache = forward_propagation(X, parameters)
        elif keep_prob < 1:
            a3, cache = forward_propagation_with_dropout(X, parameters, keep_prob)
        
        # Cost function
        if lambd == 0:
            cost = compute_cost(a3, Y)
        else:
            cost = compute_cost_with_regularization(a3, Y, parameters, lambd)
            
        # Backward propagation.
        assert(lambd==0 or keep_prob==1)    # it is possible to use both L2 regularization and dropout, 
                                            # but this assignment will only explore one at a time
        if lambd == 0 and keep_prob == 1:
            grads = backward_propagation(X, Y, cache)
        elif lambd != 0:
            grads = backward_propagation_with_regularization(X, Y, cache, lambd)
        elif keep_prob < 1:
            grads = backward_propagation_with_dropout(X, Y, cache, keep_prob)
        
        # Update parameters.
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # Print the loss every 10000 iterations
        if print_cost and i % 10000 == 0:
            print("Cost after iteration {}: {}".format(i, cost))
        if print_cost and i % 1000 == 0:
            costs.append(cost)
    
    # plot the cost
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('iterations (x1,000)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

어떤 정규화도 쓰지 않고 모델을 훈련시켜보겠습니다. 이후 훈련 세트와 테스트 세트에서의 정확도를 관찰해봅시다.

In [ ]:
parameters = model(train_X, train_Y)
print ("On the training set:")
predictions_train = predict(train_X, train_Y, parameters)
print ("On the test set:")
predictions_test = predict(test_X, test_Y, parameters)

훈련 세트 정확도는 `94.8%`인 반면, 테스트 세트의 정확도는 `91.5%`입니다. 이 모델을 **기본값 모델(baseline model)** 이라고 칭하겠습니다. 이를 통해 여러분은 정규화가 모델에 미치는 영향을 비교해볼 수 있습니다. 아래 코드를 통해서 이 모델의 decision boundary를 확인해보세요.

In [ ]:
plt.title("Model without regularization")
axes = plt.gca()
axes.set_xlim([-0.75,0.40])
axes.set_ylim([-0.75,0.65])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

정규화되지 않은 모델은 훈련 세트에 과대 적합되었음을 분명이 확인할 수 있습니다. 이제 과대 적합을 줄이기 위한 두 가지 테크닉을 살펴봅시다.

## 2. L2 Regularization ##

과(대)적합을 피하기 위한 가장 표준적인 방법은 **L2 정규화**라고 부릅니다. L2 정규화는 모델의 비용 함수를 

$$J = -\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small  y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)} \tag{1}$$
이런 형태에서

$$J_{regularized} = \small \underbrace{-\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right) \large{)} }_\text{cross-entropy cost} + \underbrace{\frac{1}{m} \frac{\lambda}{2} \sum\limits_l\sum\limits_k\sum\limits_j W_{k,j}^{[l]2} }_\text{L2 regularization cost} \tag{2}$$
이 형태로 적절히 수정하는 것으로 이루어져 있습니다. 

비용 함수를 실제로 수정해보고, 결과를 확인해봅시다.

**연습 문제**: (2)번 공식에 맞게 비용을 계산하는 `compute_cost_with_regularization()` 함수를 완성해보세요. $\sum\limits_k\sum\limits_j W_{k,j}^{[l]2}$를 계산하기 위해서는,
```python
np.sum(np.square(W1))
```
이 코드를 사용하면 됩니다.

이 작업을 $W^{[1]}$, $W^{[2]}$, $W^{[3]}$에 대해서 각각 수행해줘야 한다는 것을 잊지 마세요. 그리고 이 모든 값을 더하고 $ \frac{1}{m} \frac{\lambda}{2} $를 곱해야 합니다.

In [ ]:
# GRADED FUNCTION: compute_cost_with_regularization

def compute_cost_with_regularization(A3, Y, parameters, lambd):
    """
    Implement the cost function with L2 regularization. See formula (2) above.
    
    Arguments:
    A3 -- post-activation, output of forward propagation, of shape (output size, number of examples)
    Y -- "true" labels vector, of shape (output size, number of examples)
    parameters -- python dictionary containing parameters of the model
    
    Returns:
    cost - value of the regularized loss function (formula (2))
    """
    m = Y.shape[1]
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    W3 = parameters["W3"]
    
    cross_entropy_cost = compute_cost(A3, Y) # This gives you the cross-entropy part of the cost
    
    ### START CODE HERE ### (approx. 1 line)
    L2_regularization_cost = None
    ### END CODER HERE ###
    
    cost = cross_entropy_cost + L2_regularization_cost
    
    return cost

In [ ]:
A3, Y_assess, parameters = compute_cost_with_regularization_test_case()

print("cost = " + str(compute_cost_with_regularization(A3, Y_assess, parameters, lambd = 0.1)))

**모범 답안**: 

<table> 
    <tr>
    <td>
    <b>cost</b>
    </td>
        <td>
    1.78648594516
    </td>
    </tr>
</table> 

비용 함수를 수정했기 때문에, backward propagation(역전파) 역시 똑같이 수정해야 합니다. 모든 gradient는 새롭게 수정된 비용 함수에 맞게 새롭게 계산되어야 합니다.

**연습 문제**: 정규화에 맞게  새롭게 변경된`backwrad_propagation_with_regularization()` 함수를 구현합니다. 변경 사항은 $dW1$, $dW2$, $dW3$에만 적용됩니다. 각각에 대해 정규화된 gradient인 $\frac{d}{dW} ( \frac{1}{2}\frac{\lambda}{m}  W^2) = \frac{\lambda}{m} W$를 더해줍니다

In [ ]:
# GRADED FUNCTION: backward_propagation_with_regularization

def backward_propagation_with_regularization(X, Y, cache, lambd):
    """
    Implements the backward propagation of our baseline model to which we added an L2 regularization.
    
    Arguments:
    X -- input dataset, of shape (input size, number of examples)
    Y -- "true" labels vector, of shape (output size, number of examples)
    cache -- cache output from forward_propagation()
    lambd -- regularization hyperparameter, scalar
    
    Returns:
    gradients -- A dictionary with the gradients with respect to each parameter, activation and pre-activation variables
    """
    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    
    ### START CODE HERE ### (approx. 1 line)
    dW3 = 1./m * np.dot(dZ3, A2.T) + None
    ### END CODE HERE ###
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    ### START CODE HERE ### (approx. 1 line)
    dW2 = 1./m * np.dot(dZ2, A1.T) + None
    ### END CODE HERE ###
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    ### START CODE HERE ### (approx. 1 line)
    dW1 = 1./m * np.dot(dZ1, X.T) + None
    ### END CODE HERE ###
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,"dA2": dA2,
                 "dZ2": dZ2, "dW2": dW2, "db2": db2, "dA1": dA1, 
                 "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

In [ ]:
X_assess, Y_assess, cache = backward_propagation_with_regularization_test_case()

grads = backward_propagation_with_regularization(X_assess, Y_assess, cache, lambd = 0.7)
print ("dW1 = \n"+ str(grads["dW1"]))
print ("dW2 = \n"+ str(grads["dW2"]))
print ("dW3 = \n"+ str(grads["dW3"]))

**모범 답안**:

```
dW1 = 
[[-0.25604646  0.12298827 -0.28297129]
 [-0.17706303  0.34536094 -0.4410571 ]]
dW2 = 
[[ 0.79276486  0.85133918]
 [-0.0957219  -0.01720463]
 [-0.13100772 -0.03750433]]
dW3 = 
[[-1.77691347 -0.11832879 -0.09397446]]
```

이 모델을, $\lambda = 0.7$로 한 L2 정규화로 학습시켜보겠습니다. `model()` 함수는 기존 함수들 대신 아래의 함수들을 호출합니다.
- 기존 `compute_cost` 대신 `compute_cost_with_regularization`을 호출합니다.
- `backward_propagation` 대신 `backward_propagation_with_regularization`을 호출합니다.

In [ ]:
parameters = model(train_X, train_Y, lambd = 0.7)
print ("On the train set:")
predictions_train = predict(train_X, train_Y, parameters)
print ("On the test set:")
predictions_test = predict(test_X, test_Y, parameters)

축하합니다! 테스트 세트의 정확도가 93%까지 증가했습니다. 이정도면 프랑스 축구 협회에서도 만족하겠네요. 이제 여러분은 훈련 데이터를 더이상 과적합시키지 않습니다. 진짜 그런지 decision boundary를 그려 확인해봅시다.

In [ ]:
plt.title("Model with L2-regularization")
axes = plt.gca()
axes.set_xlim([-0.75,0.40])
axes.set_ylim([-0.75,0.65])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

**해석**:
- $\lambda$ 값은 *dev set(개발 세트)*를 사용할 때 직접 튜닝할 수 있는 하이퍼파라미터에 해당합니다.
- 위 그래프에서도 확인할 수 있듯이, L2 정규화는 decision boundary를 부드럽게 만들어줍니다. 그러나 $\lambda$ 값이 너무 클 경우, 정규화가 너무 과도하게 진행되어 모델이 큰 편향(bias)를 가지게 될 수 있습니다.

**L2 정규화가 실제로 하는 일은 무엇일까요?**:

L2 정규화는 적은 가중치를 가진 모델이 큰 가중치를 가지는 모델보다 더 간단하다는 가정에 의거합니다. 그러므로 비용 함수에 가중치의 제곱만큼 페널티를 적용해서 모든 가중치를 더 작은 값으로 유도합니다. 이는 입력이 변경됨에 따라 출력이 더 느리게 변경되는 더 부드러운 모델로 이어집니다.

<br>

**기억해야 할 것 - L2 정규화의 의미**:

- 비용 계산 
  - `cost`에 정규화 변수가 추가됩니다.
- backpropagation 함수
  - 가중치 행렬을 업데이트할 때 gradient에 추가적인 변수를 더해줍니다.
- 가중치가 점점 작아집니다. 이를 `Weight decay` 라고 부릅니다.
  - 학습 과정에서 가중치는 점점 더 작은 값으로 계산됩니다.

## 3. Dropout ##

마지막으로 **Dropout** 기법은 딥 러닝에 특화되어 널리 사용되는 정규화 기법입니다. 이 기법은 **각 반복마다 일부 뉴련을 무작위로 삭제합니다**. 아래 두 영상을 보고, 이것이 의미하는 바를 확인해보세요!

<!--
To understand drop-out, consider this conversation with a friend:
- Friend: "Why do you need all these neurons to train your network and classify images?". 
- You: "Because each neuron contains a weight and can learn specific features/details/shape of an image. The more neurons I have, the more featurse my model learns!"
- Friend: "I see, but are you sure that your neurons are learning different features and not all the same features?"
- You: "Good point... Neurons in the same layer actually don't talk to each other. It should be definitly possible that they learn the same image features/shapes/forms/details... which would be redundant. There should be a solution."
!--> 

<center>
<video width="620" height="440" src="arts/dropout1_kiank.mp4" type="video/mp4" controls>
</video>
</center>
<caption><center>그림 2: 두 번째 은닉 층에서 Dropuout 기법을 적용하는 모습.</center>

매 반복마다 $1-keep\_prob$의 확률로 특정 뉴런을 제거합니다(= 0으로 설정합니다). 제거된 뉴런은 정방향 계산과 역방향 계산 어디에도 사용되지 않습니다.

<center>
<video width="620" height="440" src="arts/dropout2_kiank.mp4" type="video/mp4" controls>
</video>
</center>
<caption><center>그림 3 : 첫 번째와 세 번째 은닉층에서 Dropout 기법을 사용하는 모습.</center></caption>

- 첫 번째 층 : 40%의 확률로 뉴런을 제거합니다.
- 세 번째 층 : 20%의 확률로 뉴런을 제거합니다.

뉴런을 제거한다면 여러분은 사실상 모델을 수정하는 것과 다름없습니다. Dropout 기법의 기본 개념은 매 반복마다 특정 층에 있는 뉴런의 일부분만 사용하는 다른 모델을 훈련한다는 것입니다. Dropout 기법을 사용하면 다른 뉴런이 언제든지 종료될 수 있으므로, 특정 뉴런의 활성화에 신경망이 덜 민감해집니다.



### 3-1. Dropout 기법에서의 Forward propagation ###

**연습 문제**: Dropout 기법을 사용해서 정방향 계산을 구현해보세요. 3개 층을 가진 인공 신경망에서, 첫 번째와 두 번째 층에 dropout을 적용해야 합니다. 입력 층과 출력 층에 대해서 dropout을 적용하진 않을겁니다.

**지시 사항**: 1, 2번 층에 있는 뉴런 중 일부를 제거해보세요. 이를 위해서 아래 4가지 스텝을 따라해야 합니다.
1. 강의에서 우리는 $a[1]$과 같은 형태를 갖는 $d[1]$(`np.random.rand()`을 사용하여 0과 1사이의 랜덤한 숫자로 구성되어 있는) 행렬을 만드는 것에 대해 논의했습니다. 이제 벡터화를 사용해서, $D^{[1]} = [d^{[1](1)} d^{[1](2)} ... d^{[1](m)}] $ 의 꼴을 가지고, $A^{[1]}$와 같은 차원을 가지는 $D{[1]}$을 만들어보겠습니다.

2. $D^{[1]}$의 각 요소를 확률(`keep_prob`)로 1로 설정하고, 그렇지 않으면 0으로 설정합니다.

    **힌트**: `keep_prop = 0.8` 이라고 가정해봅시다. 이는 각 층마다 80%의 확률로 뉴런을 유지하고, 20%의 확률로 뉴런을 제거할 것이라는걸 의미합니다. 우리는 1과 0 사이의 벡터를 만들어, 그 중 80%는 1이고 20%는 0으로 초기화하는 방법으로 이를 구현할 수 있습니다. 파이썬 코드는 아래와 같습니다
    ```python
    X = (X < keep_prob).astype(int)
    ``` 
    이 코드는 단순한 1차원 배열인 경우 아래의 if-else 구문과 동일합니다.
    ```python
    for i,v in enumerate(x):
        if v < keep_prob:
            x[i] = 1
        else: # v >= keep_prob
            x[i] = 0
    ```

    `X = (X < keep_prob).astype(int)` 과 같은 코드는 다차원 배열에서 동작합니다. 따라서 함수는 입력 배열의 형태를 그대로 유지된 상태로 결과값을 출력합니다.

    추가로, `.astype(int)`를 추가적으로 붙여주지 않으면 결과값은 boolean인 `True`와 `False`로 이루어진 배열이 됩니다. 이 boolean 값을 숫자와 곱하면, 파이썬은 자동적으로 `True`를 1로, `False`를 0으로 바꿔줍니다. 하지만, 명시적으로 데이터 타입을 표기해주는 것이 더 보기 쉬울 것이므로, `astype(int)`를 붙여주도록 하겠습니다.

3. $A^{[1]}$ 값을 $A^{[1]} * D^{[1]}$로 바꿔줍니다. 이렇게 되면, 몇개의 뉴런이 제거되는 것과 같은 효과를 냅니다. $D^{[1]}$ 행렬을 이는 다른 행렬에 곱해져서, 특정 성분을 차단시켜주는(0으로 만들어주는) 일종의 마스크로 생각해도 좋습니다. 

4. $A^{[1]}$를 `keep_prob`으로 나눕니다. 이렇게 하면 최종 cost가 dropout을 사용하지 않았을 때와 동일한 값을 유지할 수 있습니다. (이 기술은 `inverted dropout` 이라고도합니다.)

In [ ]:
# GRADED FUNCTION: forward_propagation_with_dropout

def forward_propagation_with_dropout(X, parameters, keep_prob = 0.5):
    """
    Implements the forward propagation: LINEAR -> RELU + DROPOUT -> LINEAR -> RELU + DROPOUT -> LINEAR -> SIGMOID.
    
    Arguments:
    X -- input dataset, of shape (2, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
                    W1 -- weight matrix of shape (20, 2)
                    b1 -- bias vector of shape (20, 1)
                    W2 -- weight matrix of shape (3, 20)
                    b2 -- bias vector of shape (3, 1)
                    W3 -- weight matrix of shape (1, 3)
                    b3 -- bias vector of shape (1, 1)
    keep_prob - probability of keeping a neuron active during drop-out, scalar
    
    Returns:
    A3 -- last activation value, output of the forward propagation, of shape (1,1)
    cache -- tuple, information stored for computing the backward propagation
    """
    
    np.random.seed(1)
    
    # retrieve parameters
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    
    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    ### START CODE HERE ### (approx. 4 lines)         # Steps 1-4 below correspond to the Steps 1-4 described above. 
    D1 = None                                         # Step 1: initialize matrix D1 = np.random.rand(..., ...)
    D1 = None                                         # Step 2: convert entries of D1 to 0 or 1 (using keep_prob as the threshold)
    A1 = None                                         # Step 3: shut down some neurons of A1
    A1 = None                                         # Step 4: scale the value of neurons that haven't been shut down
    ### END CODE HERE ###
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    ### START CODE HERE ### (approx. 4 lines)
    D2 = None                                         # Step 1: initialize matrix D2 = np.random.rand(..., ...)
    D2 = None                                         # Step 2: convert entries of D2 to 0 or 1 (using keep_prob as the threshold)
    A2 = None                                         # Step 3: shut down some neurons of A2
    A2 = None                                         # Step 4: scale the value of neurons that haven't been shut down
    ### END CODE HERE ###
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)
    
    cache = (Z1, D1, A1, W1, b1, Z2, D2, A2, W2, b2, Z3, A3, W3, b3)
    
    return A3, cache

In [ ]:
X_assess, parameters = forward_propagation_with_dropout_test_case()

A3, cache = forward_propagation_with_dropout(X_assess, parameters, keep_prob = 0.7)
print ("A3 = " + str(A3))

**모범 답안**: 

<table> 
    <tr>
    <td>
    <b>A3</b>
    </td>
        <td>
    [[ 0.36974721  0.00305176  0.04565099  0.49683389  0.36974721]]
    </td>
    </tr>
</table> 

### 3-2. Dropout 기법에서의 Backward propagation ###

**연습 문제**: dropout 기법을 사용한 역방향 전파를 구현합니다. 이전과 마찬가지로 3개 층을 가진 인공신경망을 훈련합니다. 캐시에 저장된 $D^{[1]}$ 및 $D^{[2]}$ 마스크를 사용하여 첫 번째 및 두 번째 은닉층에 dropout 기법을 적용합니다

**지시 사항**: Dropout 기법에서의 역방향 계산은 비교적 쉬운 편입니다. 아래의 2가지 스텝에 따라 구현해보세요.
1. 이전 정방향 계산을 구현할 때, 마스크처럼 동작하는 $D^{[1]}$ 행렬을 `A1`에 곱해주는 방식으로 뉴런 일부를 제거했었습니다. 이것과 마찬가지로 역방향 계산에선 $D^{[1]}$ 을 `dA1` 변수에 곱해주면 뉴런이 제거됩니다.

2. 정방향 계산에선, `A1`의 값을 `keep_prob` 변수로 나누어주었습니다. 역방향 계산에서도 마찬가지로 `dA1`을 `keep_prob`으로 나눠주어야 합니다. 
  - 수학적으로는, $A^{[1]}$이 `keep_prob`에 의해 스케일링되면, 파생된 미분계수인 $dA^{[1]}$ 역시 `keep_prob`에 의해 파생되어야 한다는 것을 의미합니다.

In [ ]:
# GRADED FUNCTION: backward_propagation_with_dropout

def backward_propagation_with_dropout(X, Y, cache, keep_prob):
    """
    Implements the backward propagation of our baseline model to which we added dropout.
    
    Arguments:
    X -- input dataset, of shape (2, number of examples)
    Y -- "true" labels vector, of shape (output size, number of examples)
    cache -- cache output from forward_propagation_with_dropout()
    keep_prob - probability of keeping a neuron active during drop-out, scalar
    
    Returns:
    gradients -- A dictionary with the gradients with respect to each parameter, activation and pre-activation variables
    """
    
    m = X.shape[1]
    (Z1, D1, A1, W1, b1, Z2, D2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    dA2 = np.dot(W3.T, dZ3)
    ### START CODE HERE ### (≈ 2 lines of code)
    dA2 = None              # Step 1: Apply mask D2 to shut down the same neurons as during the forward propagation
    dA2 = None              # Step 2: Scale the value of neurons that haven't been shut down
    ### END CODE HERE ###
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T)
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    ### START CODE HERE ### (≈ 2 lines of code)
    dA1 = None              # Step 1: Apply mask D1 to shut down the same neurons as during the forward propagation
    dA1 = None              # Step 2: Scale the value of neurons that haven't been shut down
    ### END CODE HERE ###
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,"dA2": dA2,
                 "dZ2": dZ2, "dW2": dW2, "db2": db2, "dA1": dA1, 
                 "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

In [ ]:
X_assess, Y_assess, cache = backward_propagation_with_dropout_test_case()

gradients = backward_propagation_with_dropout(X_assess, Y_assess, cache, keep_prob = 0.8)

print ("dA1 = \n" + str(gradients["dA1"]))
print ("dA2 = \n" + str(gradients["dA2"]))

**모범 답안**: 

```
dA1 = 
[[ 0.36544439  0.         -0.00188233  0.         -0.17408748]
 [ 0.65515713  0.         -0.00337459  0.         -0.        ]]
dA2 = 
[[ 0.58180856  0.         -0.00299679  0.         -0.27715731]
 [ 0.          0.53159854 -0.          0.53159854 -0.34089673]
 [ 0.          0.         -0.00292733  0.         -0.        ]]
```


**기억하세요**:
- Dropout 기법을 사용할 때 발생할 수 있는 **흔한 실수**는, 이 기법을 훈련과 테스팅 과정 모두에 사용한다는 것입니다. Dropout은 무작위적으로 각 층의 노드를 제거하는 방법이므로, 반드시 훈련 과정에서만 사용해야 합니다.
- [tensorflow](https://www.tensorflow.org/api_docs/python/tf/nn/dropout), [PaddlePaddle](http://doc.paddlepaddle.org/release_doc/0.9.0/doc/ui/api/trainer_config_helpers/attrs.html), [keras](https://keras.io/layers/core/#dropout), [caffe](http://caffe.berkeleyvision.org/tutorial/layers/dropout.html)와 같은 딥 러닝 프레임워크는 미리 구현된 Dropout layer 기법을 제공합니다. 이러한 프레임워크 중 일부에 대해서 곧 배울 예정이니, 이 부분에 대해서 잘 모른다고 너무 스트레스 받진 마세요!

**Dropout에 대하여 기억해야 할 것**:
- Dropout은 정규화 기법 중 하나입니다.
- Dropout 기법은 반드시 훈련 과정에서만 사용해야 합니다. 테스트에는 dropout을 사용하지 마세요.
- Dropout 기법을 쓸 때는 정방향 계산과 역전파에 모두 적용해줘야 합니다.
- 모델을 훈련하는 동안, 활성화 출력값의 기대값(평균)을 맞춰주기 위해 dropout이 적용되는 각 층을 `keep_prob` 변수로 나눠줘야 합니다. 예를 들어, `keep_prob`이 0.5라고 하면, 평균적으로 절반의 node가 제거될 것입니다. 따라서 살아있는 나머지 절반의 출력만으로도 학습이 이어질 수 있도록 출력값이 0.5만큼 나누어지는 방향으로 재조정되어야 할 것입니다. 0.5로 나눈다는 것은 2를 곱한다는 것과 같으므로, 출력값은 절반의 node가 삭제되기 이전의 기댓갑과 동일합니다.

## 4. 결론 ##

**아래는 지금까지 학습했던 모델의 성능을 비교한 표입니다**
<table> 
    <tr>
        <td>
        <b>model</b>
        </td>
        <td>
        <b>train accuracy</b>
        </td>
        <td>
        <b>test accuracy</b>
        </td>
    </tr>
        <td>
        3-layer NN without regularization
        </td>
        <td>
        95%
        </td>
        <td>
        91.5%
        </td>
    <tr>
        <td>
        3-layer NN with L2-regularization
        </td>
        <td>
        94%
        </td>
        <td>
        93%
        </td>
    </tr>
    <tr>
        <td>
        3-layer NN with dropout
        </td>
        <td>
        93%
        </td>
        <td>
        95%
        </td>
    </tr>
</table> 

정규화를 하면 할수록 훈련 세트의 정확도가 줄어듭니다. 이는 정규화 기법이 인공신경망이 훈련 세트에 과적합하는 것을 제한하기 때문입니다. 그러나 궁극적으로는 테스트 세트의 정확도가 향상되는 결과를 제공함으로, 모델 향상에 도움이 된다고 할 수 있겠습니다.

이번 과제를 마무리하신것을 축하드립니다. 여담으로, 프랑스 축구를 한 층 발전시키는 것을 자축하는 것으로 이번 과제를 끝내봅시다.

**이번 과제에서 기억해야 할 것**
- 정규화는 모델이 과적합되는 것을 줄여줍니다.
- 정규화는 각 가중치가 점점 작은 값이 되도록 합니다.
- L2 정구화와 Dropout 기법이 가장 호율적인 정규화 기법 중 하나입니다.